In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import RandomOverSampler 
from catboost import CatBoostRegressor

In [ ]:
# Proses melakukan import dataset yang diperlukan
test_set = pd.read_csv('dataset/test_feature.csv')
features = pd.read_csv('dataset/train_features.csv')
labels = pd.read_csv('dataset/train_label.csv')

In [ ]:
# Menampilkan jumlah missing value pada features
na_feature = features.isna().sum()
na_feature

In [ ]:
# Proses mengisi missing values string pada features dengan 'NULL'
features = features.fillna('NULL')

In [ ]:
features['rating_average'] = 0
features['review_count'] = 0
features['review_multiple'] = 0
for ind in features.index:
    if (features['rating'][ind] != "NULL"):
        l = []
        for t in features['rating'][ind].split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        if (len(l) == 2):
            features['rating_average'][ind] = l[0]
            features['review_count'][ind] = l[1]
            features['review_multiple'][ind] = l[0]*l[1]
        else:
            features['rating'][ind] = "NULL"
    else:
        features['rating'][ind] = "NULL"


In [ ]:
features['Restaurant']=999
features['Bar']=999
features["Gym"]=999
features['Internet']=999
features['Swimming Pools']=999

for ind in features.index:
    features['facilities'][ind] = features['facilities'][ind].lower()

    
    if (features['rating'][ind] != "NULL"):

        if 'restaurant' in features['facilities'][ind] :
            features['Restaurant'][ind] = int(1)
        else :
            features['Restaurant'][ind] = int(0)

        if 'bar' in features['facilities'][ind] :
            features['Bar'][ind] = int(1)
        else :
            features['Bar'][ind] = int(0)

        if 'swimmingpools' in features['facilities'][ind] :
            features['Swimming Pools'][ind] = int(1)
        else :
            features['Swimming Pools'][ind] = int(0)  

        if 'gym' in features['facilities'][ind] :
            features['Gym'][ind] = int(1)
        else :
            features['Gym'][ind] = int(0)  

        if 'internet' in features['facilities'][ind]  :
            features['Internet'][ind] = int(1)
        else :
            features['Internet'][ind] = int(0)

        if 'intrnet' in features['facilities'][ind]  :
            features['Internet'][ind] = int(1)
        else :
            features['Internet'][ind] = int(0)

    else :
        features['Restaurant'][ind] = int(0)
        features['Bar'][ind] = int(0)
        features['Swimming Pools'][ind] = int(0)  
        features['Gym'][ind] = int(0)  
        features['Internet'][ind] = int(0)
        
    


In [ ]:
features = pd.get_dummies(features, columns = ['location'], dtype= int)

## BATAS PENGECEKAN DATAFRAME

In [ ]:
features["Swimming Pools"].describe()

In [ ]:
features.iloc[312]

In [ ]:
for ind in labels.index:
    string = labels['Price'][ind]
    string = string.replace( 'avg/night', '')
    string = string.replace( ',', '')
    labels['Price'][ind] = float(string)
labels.head()

In [ ]:
# Drop Outlier
labels = labels.drop(index= [697,828,1030,1424,1856,2346])
features = features.drop(index= [697,828,1030,1424,1856,2346])

In [ ]:
labels = labels.astype("int32")

In [ ]:
features = features.drop(columns = ['facilities','rating'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size = 0.3, random_state = 23)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initializing
model = RandomForestRegressor(verbose=2, n_estimators= 200)

# Training
model.fit(x_train,y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_pred, y_test)


In [ ]:
print(len(y_pred))

<h1> Prediksi yang test_feature <h1>

In [ ]:
test_set.head()

In [ ]:
# Menampilkan jumlah missing value pada test_set
na_test_set = test_set.isna().sum()
na_test_set

In [ ]:
import numpy as np
test_set['rating_average'] = np.NaN
test_set['review_count'] = np.NaN
test_set['review_multiple'] = np.NaN
for ind in test_set.index:
    if (test_set['rating'][ind] != "NULL"):
        l = []
        for t in test_set['rating'][ind].split():
            try:
                l.append(float(t))
            except ValueError:
                pass
        if (len(l) == 2):
            test_set['rating_average'][ind] = l[0]
            test_set['review_count'][ind] = l[1]
            test_set['review_multiple'][ind] = l[0]*l[1]
        else:
            test_set['rating'][ind] = "NULL"
    else:
        test_set['rating'][ind] = "NULL"


print(test_set)

In [ ]:
test_set['Restaurant']=''
test_set['Bar']=''
test_set["Gym"]=''
test_set['Internet']=''
test_set['Swimming Pools']=''


In [ ]:
test_set.head()

In [ ]:
for ind in test_set.index:
    test_set['facilities'][ind] = test_set['facilities'][ind].lower()

    if 'restaurant' in test_set['facilities'][ind] :
        test_set['Restaurant'][ind] = 1
    else :
        test_set['Restaurant'][ind] = 0

    if 'bar' in test_set['facilities'][ind] :
        test_set['Bar'][ind] = 1
    else :
        test_set['Bar'][ind] = 0

    if 'swimmingpools' in test_set['facilities'][ind] :
        test_set['Swimming Pools'][ind] = 1
    else :
        test_set['Swimming Pools'][ind] = 0    

    if 'gym' in test_set['facilities'][ind] :
        test_set['Gym'][ind] = 1
    else :
        test_set['Gym'][ind] = 0   

    if 'internet' or 'intrnet' in test_set['facilities'][ind]  :
        test_set['Internet'][ind] = 1
    else :
        test_set['Internet'][ind] = 0



In [ ]:
test_set = pd.get_dummies(test_set, columns = ['location'], dtype= int)

In [ ]:
test_set = test_set.drop(columns = ['facilities','rating','ID'])

In [ ]:
test_set.head()

In [ ]:
result = model.predict(test_set)

In [ ]:
# Proses penyesuaian hasil prediksi dengan format submisi

# Proses pengubahan array menjadi dataframe
df = pd.DataFrame(result, columns=["Price"])

# Proses pengambilan respondent id original
test_set = pd.read_csv('dataset/test_feature.csv')
test_set.loc[:,"ID"]

# Proses pembuatan dataframe akhir 
df_final = test_set.ID
df_final =  pd.DataFrame(df_final, columns=["ID"])
df_final["Price"]= df["Price"] 

# Proses export dataframe menjadi csv
df_final.to_csv("prediction/bismillah_1.csv", index=False) 